<a href="https://colab.research.google.com/github/elangbijak4/infrastruktur-API/blob/main/Simple_API_Gateway_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask

In [2]:
from flask import Flask, request, jsonify
from functools import wraps
import time

In [3]:
# Initialize Flask app
app = Flask(__name__)

In [4]:
# In-memory storage for rate limiting (IP-based)
rate_limit_storage = {}
RATE_LIMIT = 5  # Max requests per minute per IP
API_KEY = "valid_api_key_12345"  # Example API Key

In [5]:
# Middleware for rate limiting
def rate_limiter(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        client_ip = request.remote_addr
        current_time = time.time()

        if client_ip not in rate_limit_storage:
            rate_limit_storage[client_ip] = []

        # Remove timestamps older than 1 minute
        rate_limit_storage[client_ip] = [t for t in rate_limit_storage[client_ip] if current_time - t < 60]

        if len(rate_limit_storage[client_ip]) >= RATE_LIMIT:
            return jsonify({"error": "Rate limit exceeded. Try again later."}), 429

        rate_limit_storage[client_ip].append(current_time)
        return f(*args, **kwargs)

    return decorated_function

In [6]:
# Middleware for API Key validation
def require_api_key(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        api_key = request.headers.get("X-API-Key")
        if api_key != API_KEY:
            return jsonify({"error": "Invalid or missing API Key."}), 401
        return f(*args, **kwargs)

    return decorated_function

In [7]:
# Backend services
@app.route("/service-a", methods=["GET"])
def service_a():
    return jsonify({"message": "Welcome to Service A!"})

@app.route("/service-b", methods=["POST"])
def service_b():
    data = request.json
    transformed_data = {"received": data, "processed": True}
    return jsonify(transformed_data)

In [8]:
# API Gateway
@app.route("/api/<service>", methods=["GET", "POST"])
@rate_limiter
@require_api_key
def api_gateway(service):
    if service == "service-a":
        return service_a()
    elif service == "service-b":
        return service_b()
    else:
        return jsonify({"error": "Service not found."}), 404

In [9]:
# Logging all requests
@app.before_request
def log_request():
    print(f"Incoming request: {request.method} {request.path} from {request.remote_addr}")

In [ ]:
# Run the Flask app
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
